# KGAT Inference & Explanation

This notebook loads the trained KGAT model and provides explanations for recommendations.

In [ ]:
import torch
from src.model.kgat import KGAT
from src.model.explainer import KGATExplainer

# Load Model
# model = KGAT(...) 
# model.load_state_dict(torch.load('model.pth'))

In [ ]:
explainer = KGATExplainer(model)
user_id = 0
item_id = 10
explanation = explainer.explain(g, user_id, item_id)
print(explanation)
explainer.visualize(explanation)